In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [21]:
# parameters
tensorboard_dir = '../tb/run4';
tensorboardFlag = True;
workers = 10;

batchSize = 64;
epochs = 5;
validation_size=0.3;

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
df_train = pd.read_csv('../input/labels.csv')
df_test = pd.read_csv('../input/sample_submission.csv')

print('Training images: ',df_train.shape[0])
print('Test images: ',df_test.shape[0])

# reduce dimensionality
#df_train = df_train.head(100)
#df_test = df_test.head(100)

Training images:  10222
Test images:  10357


In [4]:
df_train.head(10)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
im_size = 90

In [8]:
x_train = []
y_train = []
x_test = []

In [9]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = cv2.imread('../input/train/{}.jpg'.format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1

100%|██████████| 10222/10222 [00:31<00:00, 328.85it/s]


In [10]:
for f in tqdm(df_test['id'].values):
    img = cv2.imread('../input/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))

100%|██████████| 10357/10357 [00:30<00:00, 337.61it/s]


In [11]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

We check the shape of the outputs to make sure everyting went as expected.

In [12]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

(10222, 90, 90, 3)
(10222, 120)
(10357, 90, 90, 3)


We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [13]:
num_class = y_train_raw.shape[1]
print('Number of classes: ', num_class)

Number of classes:  120


It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [14]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=validation_size, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [15]:
# Create the base pre-trained model
# Can't download weights in the kernel
base_model = VGG19(#weights='imagenet',
    weights = 'imagenet', include_top=False, input_shape=(im_size, im_size, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
predictions = Dense(num_class, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

callbacks_list = [];
callbacks_list.append(keras.callbacks.EarlyStopping(
    monitor='val_acc',
    patience=10,
    verbose=1));
if tensorboardFlag:
    callbacks_list.append(keras.callbacks.TensorBoard(
            log_dir=tensorboard_dir,
            histogram_freq=5,
            write_graph=False,
            write_images=False));
    print('Tensorboard activated')
else:
    print('Tensorboard NOT activated')


model.summary()

Tensorboard activated
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 90, 90, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 90, 90, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 90, 90, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 45, 45, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 45, 45, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 45, 45, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 22, 22, 128)      

In [16]:
def generator(X,Y,batch_size):
    batch_features = np.ndarray(shape=(batch_size,) + X.shape[1:],
                                dtype=X.dtype);
    batch_labels = np.ndarray(shape=(batch_size,) + Y.shape[1:],
                                dtype=Y.dtype)
    N = X.shape[0];
    
    while True:
        for i in range(batch_size):
            # choose random index in features
            index= np.random.choice(N,1)
            batch_features[i] = X[index]
            batch_labels[i] = Y[index]
        yield batch_features, batch_labels

In [22]:
#model.fit(X_train, Y_train,
#          epochs=epochs,
#          batch_size = batchSize,
#          validation_data=(X_valid, Y_valid),
#          verbose=1,
#          callbacks=callbacks_list)
steps_per_epoch = round(X_train.shape[0]/batchSize)

model.fit_generator(generator(X_train,Y_train,batchSize),
                   steps_per_epoch=steps_per_epoch,
                   epochs=epochs,
                   verbose=1,
                   callbacks=callbacks_list,
                   validation_data=(X_valid,Y_valid),
                   workers=workers,
                   use_multiprocessing=True)

/home/mattia-lecci/venv_project/lib/python3.5/site-packages/keras/engine/training.py:2057: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/5
112/112 [==============================] - 326s 3s/step - loss: 2.9173 - acc: 0.3749 - val_loss: 4.0691 - val_acc: 0.1187
Epoch 2/5
112/112 [==============================] - 27s 238ms/step - loss: 2.6647 - acc: 0.4325 - val_loss: 4.0375 - val_acc: 0.1141
Epoch 3/5
112/112 [==============================] - 27s 237ms/step - loss: 2.4733 - acc: 0.4831 - val_loss: 4.0382 - val_acc: 0.1089
Epoch 4/5
112/112 [==============================] - 27s 237ms/step - loss: 2.2769 - acc: 0.5307 - val_loss: 4.0325 - val_acc: 0.1291
Epoch 5/5
112/112 [==============================] - 27s 237ms/step - loss: 2.1328 - acc: 0.5686 - val_loss: 4.0207 - val_acc: 0.1275


Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [23]:
preds = model.predict(x_test, verbose=1)

10357/10357 [==============================] - 29s 3ms/step


In [24]:
sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', df_test['id'])
sub.head(10)

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,7.283219e-06,5.159800e-03,1.680854e-05,0.000019,0.003696,0.000145,0.000060,0.001805,2.860163e-02,...,4.710769e-04,0.000434,6.021040e-05,5.213224e-04,3.486446e-02,0.215926,0.000001,3.744449e-03,0.000107,0.000034
1,00102ee9d8eb90812350685311fe5890,5.072772e-04,3.275320e-03,1.299841e-04,0.000086,0.000788,0.000133,0.001052,0.000211,3.616280e-04,...,5.975930e-03,0.007185,1.706774e-05,6.517411e-05,9.465597e-04,0.000892,0.004254,7.930796e-04,0.004599,0.000427
2,0012a730dfa437f5f3613fb75efcd4ce,3.399096e-04,1.962730e-03,5.590480e-04,0.010398,0.002965,0.016164,0.000981,0.045377,6.086941e-03,...,8.006244e-04,0.007229,4.533662e-04,4.399455e-03,1.497823e-02,0.006038,0.005092,2.784549e-02,0.004651,0.001340
3,001510bc8570bbeee98c8d80c8a95ec1,3.256522e-03,8.096942e-03,7.001467e-04,0.019202,0.002568,0.016908,0.003874,0.044180,2.581268e-04,...,4.127896e-04,0.000930,1.245752e-04,8.357690e-05,7.694110e-04,0.000778,0.000078,1.847809e-02,0.000520,0.001965
4,001a5f3114548acdefa3d4da05474c2e,4.359524e-03,8.033515e-03,5.733996e-03,0.021665,0.008522,0.001473,0.010801,0.000860,4.135124e-03,...,8.335645e-04,0.000628,2.454243e-04,1.543871e-04,1.210303e-02,0.000174,0.000352,5.828367e-03,0.003394,0.000966
5,00225dcd3e4d2410dd53239f95c0352f,6.357942e-06,2.115214e-02,9.138526e-05,0.000639,0.000652,0.000005,0.001635,0.002837,1.379738e-02,...,2.282886e-03,0.001823,1.587006e-04,7.666910e-03,8.481338e-03,0.008367,0.000227,3.740757e-02,0.001545,0.000319
6,002c2a3117c2193b4d26400ce431eebd,2.760198e-03,1.036627e-02,4.684896e-03,0.008145,0.002332,0.004590,0.056360,0.009223,4.053432e-04,...,8.365962e-04,0.013964,1.291825e-03,2.209602e-03,2.325887e-03,0.016920,0.006088,8.215307e-04,0.003276,0.029907
7,002c58d413a521ae8d1a5daeb35fc803,5.354577e-08,4.390910e-07,2.701080e-09,0.000001,0.000193,0.000001,0.000763,0.000125,5.770060e-08,...,6.654978e-07,0.000002,1.552045e-11,4.057651e-08,2.089024e-10,0.000003,0.077765,2.873224e-07,0.023920,0.000067
8,002f80396f1e3db687c5932d7978b196,7.752546e-04,1.544805e-03,2.398138e-02,0.013118,0.006337,0.001520,0.005562,0.002427,2.008065e-03,...,4.853016e-04,0.000421,3.891230e-04,3.111624e-04,1.954083e-02,0.003258,0.000946,1.096395e-02,0.000953,0.001894
9,0036c6bcec6031be9e62a257b1c3c442,5.019853e-04,5.308604e-04,8.655255e-04,0.000174,0.001329,0.002169,0.000409,0.000207,2.069571e-03,...,6.965097e-05,0.000121,2.512516e-04,1.849740e-03,3.058268e-01,0.001205,0.000011,2.365966e-04,0.000115,0.000252
